This notebook allow you to print results of models in tables

In [1]:
from systemtools.hayj import *
from systemtools.location import *
from systemtools.basics import *
from systemtools.file import *
from systemtools.printer import *
from twinews.utils import *
from twinews.evaluation.utils import *
from twinews.models.ranking import *

In [2]:
if False:
    purgeSubsampledRankings()

In [3]:
if False:
    removeRankingsAndScores('tfidf-b0e81')

In [4]:
def basicPrint(model, *args, b=None, w=None, splitVersion=2, **kwargs):
    if w is not None and isinstance(w, str):
        w = [w]
    if b is not None and isinstance(b, str):
        b = [b]
    if w is not None and len(args) > 0:
        args = list(args) + list(w)
    printReportKwargs = \
    {
        'model': model,
        'splitVersion': splitVersion,
        'noSubsampling': True,
        'metaFilter': \
        {
            # "implementation": "gensim-lda",
        },
        # 'allowedMetrics': {'ndcg', 'div@100', 'topic-div@100', 'swjacc-div@100', 'jacc-div@100'}, # None, {'ndcg', 'ndcg@10'}
        'discardedMetrics': {'map', 'mrr', 'ndcg@10', 'ndcg@100', 'p@10', 'p@100'},
        'whiteMetricPatterns': None if len(args) == 0 else set(list(args) + ['ndcg']),
        'blackMetricPatterns': b,
        'sortBy': 'ndcg',
    }
    printReportKwargs = mergeDicts(printReportKwargs, kwargs)
    printReport(**printReportKwargs)

In [5]:
def printAll(*args, **kwargs):
    kwargs['onlyFields'] = {'id', 'model'}
    kwargs['excludedModels'] = {'combin', 'ideal', 'worst', 'random'}
    return basicPrint(None, *args, **kwargs)

# Misc

In [ ]:
if False:
    twinewsRankings = getTwinewsRankings()
    keysToDelete = set()
    for key in twinewsRankings.keys():
        if "combin" in key:
            meta = twinewsRankings.getMeta(key)
            hasTFIDF = False
            for m in meta['models']:
                if 'tfidf' in m:
                    hasTFIDF = True
                    break
            if hasTFIDF and meta['splitVersion'] == 1:
                keysToDelete.add(key)
    print(keysToDelete)

In [ ]:
if False:
    for key in keysToDelete:
        removeRankingsAndScores(key)

In [ ]:
# Removing combin models with maxUsers is not None:
if False:
    twinewsRankings = getTwinewsRankings()
    ids = twinewsRankings.keys()
    bp(ids)
    for currentId in ids:
        if 'combin' in currentId:
            meta = twinewsRankings.getMeta(currentId)
            if meta['maxUsers'] is not None:
                del twinewsRankings[currentId]

# Model lists per split version

In [ ]:
def printModelList(splitVersion):
    twinewsRankings = getTwinewsRankings()
    models = set()
    for key in twinewsRankings.keys():
        meta = twinewsRankings.getMeta(key)
        if meta["splitVersion"] == splitVersion:
            models.add(meta['model'])
    print(" ".join(sorted(list(models))) + " (" + str(len(models)) + ")")

In [ ]:
printModelList(2)

In [ ]:
printModelList(1)

# Reference models

In [14]:
basicPrint('ideal', splitVersion=2)
basicPrint('random', splitVersion=2)
basicPrint('worst', splitVersion=2)

Normalized metrics: {'div@100', 'topic-div@100', 'snov@100', 'style-nov@100', 'nov@100', 'topic-snov@100', 'style-div@100', 'topic-nov@100', 'style-snov@100'}
serializabledict twinews-dominances initialised.


,id,maxUsers,model,splitVersion,ndcg,div@100,topic-div@100,jacc-div@100,swjacc-div@100,style-div@100,nov@100,topic-nov@100,style-nov@100,snov@100,topic-snov@100,style-snov@100,tfidf-ser@100,wtfidf-ser@100,bm25-ser@100,jacc-ser@100,style-ser@100
0,ideal-5ac2d,None,ideal,2,1,0.8848,0.73305,0.89268,0.97591,0.62458,0.86265,0.70715,0.61526,0.7817,0.770133,0.69584,1,1,1,1,1


These values are common to all rows (10):

	- model: random
	- maxUsers: None
	- splitVersion: 2
Normalized metrics: {'div@100', 'topic-div@100', 'snov@100', 'style-nov@100', 'nov@100', 'topic-snov@100', 'style-div@100', 'topic-nov@100', 'style-snov@100'}
serializabledict twinews-dominances initialised.


,id,seed,ndcg,div@100,topic-div@100,jacc-div@100,swjacc-div@100,style-div@100,nov@100,topic-nov@100,jacc-nov@100,swjacc-nov@100,style-nov@100,snov@100,topic-snov@100,jacc-snov@100,swjacc-snov@100,style-snov@100,tfidf-ser@100,wtfidf-ser@100,bm25-ser@100,jacc-ser@100,style-ser@100
9,random-f03b8,4,0.22153,0.8834,0.73575,0.89232,0.97522,0.63244,0.8695,0.727975,nan,nan,0.62658,0.796433,0.7945,nan,nan,0.70442,0.16297,0.16332,0.16167,0.16435,0.16603
3,random-4d05f,8,0.21667,0.8837,0.735775,0.89261,0.97528,0.63746,0.87135,0.7322,nan,nan,0.63076,0.7998,0.799583,nan,nan,0.70996,0.00746,0.00725,0.00763,0.00558,0.00712
6,random-6fae4,3,0.21645,0.88175,0.73255,0.89184,0.97483,0.63516,0.87025,0.730625,nan,nan,0.62892,0.7988,0.797867,nan,nan,0.70452,0.00155,0.00157,0.00165,0.00115,0.00126
1,random-04de9,2,0.21099,0.8829,0.734225,0.89187,0.97506,0.63052,0.87105,0.732,0.89083,0.97286,0.62852,0.799533,0.799417,0.85758,0.95055,0.70858,0.00192,0.00174,0.00185,0.00144,0.00164
5,random-a9655,1,0.21023,0.88165,0.73235,0.89224,0.97489,0.63694,0.86915,0.727575,nan,nan,0.62702,0.796667,0.793933,nan,nan,0.70044,0.08355,0.08332,0.08431,0.07592,0.08164
4,random-21e0b,6,0.20917,0.88415,0.73665,0.89261,0.97536,0.63596,0.87135,0.732125,0.89122,0.973,0.63014,0.799533,0.799433,0.85799,0.95072,0.70922,0.01489,0.0156,0.01545,0.01258,0.01425
2,random-d87ed,9,0.20841,0.8815,0.73115,0.89179,0.97478,0.64014,0.8702,0.7301,nan,nan,0.63184,0.799,0.797467,nan,nan,0.7081,0.00027,0.00022,0.00029,0.00023,0.00029
7,random-a1ece,7,0.20738,0.8829,0.7356,0.89235,0.9751,0.63802,0.87075,0.731675,nan,0.97285,0.63086,0.798967,0.798833,nan,0.95049,0.70942,0.01383,0.01397,0.01407,0.01257,0.01366
0,random-093c0,0,0.20427,0.88345,0.73665,0.89238,0.97523,0.63288,0.87075,0.7315,nan,nan,0.62856,0.798533,0.798317,nan,nan,0.70736,0.04008,0.03966,0.04024,0.03515,0.03969
8,random-e27ab,5,0.20295,0.8832,0.7353,0.89251,nan,0.6347,0.87015,0.72955,nan,nan,0.62722,0.797567,0.79575,nan,0.9504,0.70248,0.05365,0.05325,0.05396,0.04752,0.05309


Normalized metrics: {'div@100', 'topic-div@100', 'snov@100', 'style-nov@100', 'nov@100', 'topic-snov@100', 'style-div@100', 'topic-nov@100', 'style-snov@100'}
serializabledict twinews-dominances initialised.


,id,maxUsers,model,splitVersion,ndcg,div@100,topic-div@100,jacc-div@100,swjacc-div@100,style-div@100,nov@100,topic-nov@100,style-nov@100,snov@100,topic-snov@100,jacc-snov@100,style-snov@100,tfidf-ser@100,wtfidf-ser@100,bm25-ser@100,jacc-ser@100,style-ser@100
0,worst-f3f99,None,worst,2,0.18191,0.8885,0.737725,0.89301,0.97643,0.62538,0.8755,0.738025,0.63278,0.8044,0.805167,0.85897,0.7236,0,0,0,0,0


# Ad-hoc models

In [ ]:
basicPrint('jaccard', 'div', b=['topic', 'jacc', 'style'])

In [13]:
basicPrint('bm25')

These values are common to all rows (26):

	- epsilon: 0.25
	- model: bm25
	- maxUsers: None
	- splitVersion: 2
Normalized metrics: {'div@100', 'topic-div@100', 'snov@100', 'style-nov@100', 'nov@100', 'topic-snov@100', 'style-div@100', 'topic-nov@100', 'style-snov@100'}
serializabledict twinews-dominances initialised.


,id,b,doLemmatization,k1,lowercase,maxDF,minDF,ndcg,div@100,topic-div@100,jacc-div@100,swjacc-div@100,style-div@100,nov@100,topic-nov@100,jacc-nov@100,swjacc-nov@100,style-nov@100,snov@100,topic-snov@100,jacc-snov@100,swjacc-snov@100,style-snov@100,tfidf-ser@100,wtfidf-ser@100,bm25-ser@100,jacc-ser@100,style-ser@100
21,bm25-1eb2a,1.2,False,2.4,False,300,0.0005,0.55949,0.7658,0.501075,0.87403,0.95734,0.52684,0.7652,0.458575,nan,nan,0.51148,0.6334,0.441217,nan,nan,0.52742,0.33034,0.24886,0,0.65517,0.51137
2,bm25-e8059,1.2,False,2.6,False,300,0.0005,0.5592,0.7699,0.50285,0.87471,0.95822,0.53056,0.76695,0.4589,nan,nan,0.51322,0.634567,0.439567,nan,nan,0.52978,0.33449,0.25983,0.01812,0.65945,0.51385
17,bm25-efa94,0.9,False,2.4,False,300,0.0005,0.54458,0.7331,0.499325,0.86953,0.95017,0.50316,0.75355,0.4665,nan,nan,0.50322,0.6273,0.465033,nan,nan,0.51622,0.27842,0.13436,0.07415,0.59759,0.47698
9,bm25-f3052,0.9,False,2.2,False,300,0.0005,0.54253,0.73085,0.4994,0.86926,0.94963,0.50166,0.7529,0.46765,nan,nan,0.50298,0.627433,0.468,nan,nan,0.51604,0.27386,0.12728,0.07611,0.59166,0.47301
16,bm25-5481b,0.9,False,2,False,300,0.0005,0.5398,0.72835,0.499725,0.86895,0.94903,0.50008,0.75225,0.469175,nan,nan,0.50278,0.627667,0.4716,nan,nan,0.51596,0.26955,0.11964,0.07967,0.58489,0.46869
11,bm25-9e342,1.4,False,2.4,False,300,0.0005,0.53963,0.80665,0.529375,0.88249,nan,0.56518,0.78435,0.470675,nan,nan,0.5324,0.651867,0.444267,nan,nan,0.5585,0.35443,0.32041,0.11722,0.67534,0.52116
22,bm25-87073,0.75,False,2.4,False,300,0.0005,0.52998,0.72115,0.503725,0.8682,0.94731,0.49726,0.75065,0.475175,0.87801,nan,0.50328,0.627833,0.481683,0.83454,nan,0.51684,0.25264,0.0949,0.08955,0.56396,0.45621
5,bm25-f55be,0.75,False,2.2,False,300,0.0005,0.52757,0.7195,0.5041,0.86799,0.94689,0.49638,0.75035,0.47655,nan,nan,0.5034,0.628233,0.484683,nan,nan,0.51718,0.24965,0.09096,0.09043,0.55928,0.45303
4,bm25-2c4b0,1.4,False,2.6,False,300,0.0005,0.52735,0.8184,0.542975,0.88528,0.96804,0.57672,0.7905,0.47785,nan,nan,0.53964,0.658767,0.450717,nan,nan,0.5696,0.35145,0.32455,0.13153,0.67317,0.51792
10,bm25-a91ce,0.75,False,2,False,300,0.0005,0.52465,0.71775,0.5046,0.86777,0.94643,0.49544,0.75005,0.478225,0.87831,0.95623,0.50358,0.628733,0.488133,0.83456,0.9192,0.51766,0.24545,0.08637,0.09158,0.55341,0.44897


In [ ]:
basicPrint('BM25_yf', noSubsampling=False)

# Models that use vector representations

In [9]:
basicPrint('lda')

These values are common to all rows (18):

	- model: lda
	- lowercase: True
	- ldaLearningMethod: online
	- maxUsers: None
	- useExtraNews: False
	- ldaLearningDecay: 0.7
	- splitVersion: 2
	- maxDF: 300
Normalized metrics: {'div@100', 'topic-div@100', 'snov@100', 'style-nov@100', 'nov@100', 'topic-snov@100', 'style-div@100', 'topic-nov@100', 'style-snov@100'}
serializabledict twinews-dominances initialised.


,id,distance,doLemmatization,historyRef,implementation,ldaLearningOffset,maxDocuments,maxIter,minDF,nbTopics,useTFIDF,ndcg,div@100,topic-div@100,jacc-div@100,swjacc-div@100,style-div@100,nov@100,topic-nov@100,jacc-nov@100,swjacc-nov@100,style-nov@100,snov@100,topic-snov@100,jacc-snov@100,swjacc-snov@100,style-snov@100,tfidf-ser@100,wtfidf-ser@100,bm25-ser@100,jacc-ser@100,style-ser@100
17,lda-d4f1d,cosine,False,30,gensim-lda,1,300000,60,0.0005,100,False,0.41897,0.8147,0.474225,0.8836,0.96685,0.55934,0.8002,0.46965,nan,nan,0.5422,0.689033,0.4505,nan,nan,0.58282,0.2979,0.28661,0.22176,0.56964,0.44436
12,lda-b123f,cosine,False,30,gensim-lda,1,100000,120,0.0005,100,False,0.41415,0.8131,0.479975,0.88368,0.96666,0.55584,0.80065,0.478775,nan,nan,0.5411,0.690867,0.46385,nan,nan,0.58098,0.30252,0.28122,0.22199,0.55546,0.43829
13,lda-d1220,cosine,False,0.3,gensim-lda,1,100000,60,0.0005,100,False,0.38648,0.8151,0.4755,0.88368,0.9673,0.55446,0.80505,0.4922,nan,nan,0.54522,0.6916,0.460517,nan,nan,0.58618,0.27006,0.26042,0.20491,0.52617,0.41328
7,lda-af807,cosine,False,30,gensim-lda,1,10000,60,0.0005,100,False,0.37941,0.80545,0.4898,0.88208,0.96476,0.5482,0.80325,0.512375,nan,nan,0.54612,0.700733,0.518217,nan,nan,0.5874,0.24019,0.20805,0.16835,0.45287,0.35965
15,lda-759de,cosine,False,0.6,gensim-lda,1,10000,60,0.0005,100,False,0.37484,0.81185,0.497825,0.88227,0.96573,0.55976,0.80685,0.51785,0.88271,nan,0.55152,0.7067,0.524933,0.84532,0.93728,0.59532,0.25086,0.21656,0.17722,0.45861,0.36596
11,lda-64d0c,cosine,False,1.0,gensim-lda,1,10000,60,0.0005,100,False,0.37478,0.7978,0.476175,0.87916,0.96317,0.54388,0.80225,0.5177,nan,0.96427,0.54562,0.702567,0.530883,nan,0.93575,0.58696,0.23453,0.19799,0.16511,0.43699,0.34313
5,lda-2fa8e,euclidean,False,0.3,gensim-lda,1,10000,60,0.0005,100,False,0.36168,0.80485,0.4915,0.88171,0.96346,0.53932,0.8051,0.52215,nan,0.96438,0.54362,0.708867,0.536533,nan,0.93551,0.5829,0.23992,0.20328,0.18477,0.40936,0.32674
2,lda-b849e,cosine,False,0.3,gensim-lda,1,10000,60,0.0005,300,False,0.35651,0.8051,0.5159,0.88224,0.9645,0.55262,0.80665,0.539475,nan,nan,0.55224,0.700067,0.535717,nan,nan,0.59738,0.20224,0.18566,0.15744,0.40848,0.32529
10,lda-7786b,cosine,True,0.3,gensim-lda,1,10000,60,0.0005,100,False,0.35505,0.81315,0.518075,0.88137,0.96584,0.57208,0.8095,0.534925,nan,nan,0.5623,0.703367,0.531717,nan,nan,0.61262,0.20573,0.19357,0.15901,0.4282,0.32902
3,lda-d3b82,cosine,False,0.3,gensim-lda,1,10000,60,0.0005,100,False,0.35335,0.8195,0.519275,0.88448,0.96705,0.5638,0.8127,0.5382,nan,nan,0.55672,0.710967,0.536217,nan,nan,0.59968,0.22421,0.20693,0.17618,0.42019,0.32978


In [10]:
basicPrint('nmf')

These values are common to all rows (19):

	- distance: cosine
	- model: nmf
	- nmfAlpha: 0.1
	- lowercase: True
	- implementation: sklearn-nmf
	- maxUsers: None
	- useExtraNews: False
	- minDF: 0.0005
	- splitVersion: 2
	- maxDF: 300
Normalized metrics: {'div@100', 'topic-div@100', 'snov@100', 'style-nov@100', 'nov@100', 'topic-snov@100', 'style-div@100', 'topic-nov@100', 'style-snov@100'}
serializabledict twinews-dominances initialised.


,id,doLemmatization,historyRef,maxDocuments,maxIter,nbTopics,nmfInit,nmfL1Ratio,ndcg,div@100,topic-div@100,jacc-div@100,swjacc-div@100,style-div@100,nov@100,topic-nov@100,jacc-nov@100,swjacc-nov@100,style-nov@100,snov@100,topic-snov@100,jacc-snov@100,swjacc-snov@100,style-snov@100,tfidf-ser@100,wtfidf-ser@100,bm25-ser@100,jacc-ser@100,style-ser@100
11,nmf-9cd4f,False,30,100000,200,100,nndsvd,0,0.49322,0.79755,0.403625,0.88125,0.9633,0.53998,0.7838,0.393225,nan,nan,0.5203,0.675233,0.394367,nan,nan,0.55332,0.37143,0.32401,0.25758,0.62312,0.50439
5,nmf-330c8,False,30,10000,200,100,nndsvd,0,0.48901,0.79935,0.410075,0.88119,0.96368,0.54224,0.78475,0.398075,nan,nan,0.5216,0.6752,0.3952,nan,nan,0.55498,0.37164,0.32492,0.25487,0.62502,0.50331
6,nmf-b1354,False,30,100000,200,100,None,0,0.48842,0.7977,0.405175,0.88081,0.96328,0.54184,0.78385,0.39485,0.88032,0.96258,0.52166,0.675167,0.395833,0.84117,0.93237,0.5552,0.37195,0.32504,0.26062,0.62284,0.50519
9,nmf-ee6c6,False,1.0,100000,200,100,nndsvd,0,0.48682,0.79675,0.4015,0.88075,0.96303,0.54176,0.78335,0.392525,nan,nan,0.52188,0.6764,0.399067,nan,nan,0.55598,0.36945,0.31562,0.24941,0.61861,0.49934
0,nmf-ad87b,False,30,100000,400,100,nndsvd,0,0.48653,0.7977,0.4042,0.88088,0.9633,0.54304,0.78385,0.393675,0.88038,nan,0.52252,0.6754,0.394483,0.84128,nan,0.55648,0.36906,0.31956,0.25251,0.62007,0.50228
1,nmf-84f4b,False,0.6,100000,200,100,nndsvd,0,0.48336,0.80095,0.415075,0.88127,0.96395,0.54588,0.7853,0.39865,nan,nan,0.52382,0.673233,0.380617,nan,nan,0.5566,0.36531,0.32456,0.25455,0.62547,0.50224
18,nmf-054a5,False,10,100000,200,100,nndsvd,0,0.4805,0.80245,0.4187,0.88157,0.96432,0.5467,0.7865,0.4014,nan,nan,0.52476,0.670733,0.370617,nan,nan,0.55752,0.3575,0.33049,0.26015,0.62552,0.50739
2,nmf-87941,False,0.3,100000,400,100,nndsvd,0,0.47088,0.8074,0.44175,0.88259,0.96535,0.55308,0.78865,0.411,nan,nan,0.52824,0.665767,0.33985,nan,nan,0.5589,0.34288,0.32677,0.25575,0.62355,0.50489
4,nmf-b286d,False,0.3,300000,500,100,nndsvd,0,0.46988,0.80855,0.44,0.8827,0.96555,0.55402,0.78915,0.40985,nan,nan,0.52912,0.666267,0.33835,nan,nan,0.56022,0.33846,0.32581,0.25365,0.62117,0.50245
12,nmf-2d0d6,False,0.3,300000,400,100,nndsvd,0,0.46935,0.8085,0.441025,0.88276,0.96556,0.55484,0.78915,0.410625,0.88133,nan,0.52968,0.666267,0.3394,0.84195,nan,0.56138,0.3374,0.3244,0.25245,0.62113,0.50243


In [11]:
basicPrint('tfidf')

Found key difference: {'maxIter'}
Found key difference: {'maxIter'}
Found key difference: {'maxIter'}
Found key difference: {'maxIter'}
Found key difference: {'maxIter'}
Found key difference: {'maxIter'}
Found key difference: {'maxIter'}
Found key difference: {'maxIter'}
Found key difference: {'maxIter'}
Found key difference: {'maxIter'}
Found key difference: {'maxIter'}
Found key difference: {'maxIter'}
These values are common to all rows (22):

	- distance: cosine
	- model: tfidf
	- maxUsers: None
	- sublinearTF: True
	- splitVersion: 2
Normalized metrics: {'div@100', 'topic-div@100', 'snov@100', 'style-nov@100', 'nov@100', 'topic-snov@100', 'style-div@100', 'topic-nov@100', 'style-snov@100'}
serializabledict twinews-dominances initialised.


,id,dimensions,doLemmatization,historyRef,lowercase,maxDF,maxIter,minDF,ndcg,div@100,topic-div@100,jacc-div@100,swjacc-div@100,style-div@100,nov@100,topic-nov@100,jacc-nov@100,swjacc-nov@100,style-nov@100,snov@100,topic-snov@100,jacc-snov@100,swjacc-snov@100,style-snov@100,tfidf-ser@100,wtfidf-ser@100,bm25-ser@100,jacc-ser@100,style-ser@100
4,tfidf-be107,nan,False,0.4,False,300,N/A,0.0005,0.54867,0.7172,0.51225,0.87037,0.94769,0.50044,0.74765,0.476875,nan,nan,0.50406,0.613267,0.4601,nan,nan,0.52124,0.23261,0.12131,0.14043,0.60292,0.48854
18,tfidf-6c83e,nan,False,0.35,False,300,N/A,0.0005,0.54863,0.72005,0.515175,0.87099,0.94834,0.50258,0.7487,0.478,0.87857,nan,0.505,0.610733,0.45465,0.8334,nan,0.52186,0.22631,0.13644,0.14657,0.606,0.49082
3,tfidf-01001,nan,False,0.3,False,300,N/A,0.0005,0.5484,0.7226,0.5182,0.87153,0.94892,0.50442,0.74965,0.47935,nan,nan,0.50592,0.6084,0.449967,nan,nan,0.52256,0.21799,0.14702,0.15087,0.60719,0.49254
8,tfidf-7c894,nan,False,0.45,False,300,N/A,0.0005,0.54769,0.7156,0.510725,0.87,0.9473,0.49922,0.74715,0.476525,nan,nan,0.5036,0.614933,0.463783,nan,nan,0.52106,0.23659,0.11081,0.13673,0.60078,0.48651
13,tfidf-f57a2,nan,False,0.5,False,300,N/A,0.0005,0.54631,0.7134,0.508775,0.86948,0.94676,0.49758,0.7465,0.476175,0.87845,nan,0.50296,0.617133,0.468867,0.83338,nan,0.5208,0.241,0.09901,0.13257,0.59816,0.4827
14,tfidf-11f90,nan,False,0.55,False,300,N/A,0.0005,0.54509,0.71255,0.5081,0.86925,0.94652,0.49698,0.74625,0.476225,nan,nan,0.5028,0.618167,0.471317,nan,nan,0.52084,0.24158,0.09071,0.13139,0.59598,0.48101
9,tfidf-9bb25,nan,False,0.6,False,300,N/A,0.0005,0.54388,0.7112,0.507225,0.86892,0.94617,0.49602,0.7459,0.4764,nan,nan,0.50252,0.619633,0.474983,nan,nan,0.52088,0.24364,0.08019,0.12969,0.59356,0.47846
12,tfidf-e85ec,nan,False,0.2,False,300,N/A,0.0005,0.54294,0.73125,0.52945,0.87322,0.95073,0.51092,0.7534,0.485825,nan,nan,0.50948,0.6022,0.439067,nan,nan,0.52586,0.19125,0.17189,0.16082,0.60699,0.49423
11,tfidf-6056f,nan,False,30,False,300,N/A,0.0005,0.53276,0.7059,0.50585,0.86746,0.94462,0.49226,0.74505,0.479975,nan,nan,0.50208,0.625233,0.491617,nan,nan,0.52198,0.24508,0.0249,0.12804,0.5773,0.46582
20,tfidf-487b6,3000,False,0.3,False,300,10,0.0005,0.53182,0.7279,0.4889,0.87247,0.94927,0.50166,0.75235,0.46375,0.87974,0.95719,0.50508,0.6178,0.433333,0.83451,0.91886,0.52248,0.23706,0.16834,0.15748,0.5979,0.48511


In [12]:
basicPrint('dbert-ft')

These values are common to all rows (16):

	- model: dbert-ft
	- maxUsers: None
	- splitVersion: 2
Normalized metrics: {'div@100', 'topic-div@100', 'snov@100', 'style-nov@100', 'nov@100', 'topic-snov@100', 'style-div@100', 'topic-nov@100', 'style-snov@100'}
serializabledict twinews-dominances initialised.


,id,distance,historyRef,ndcg,div@100,topic-div@100,jacc-div@100,swjacc-div@100,style-div@100,nov@100,topic-nov@100,jacc-nov@100,swjacc-nov@100,style-nov@100,snov@100,topic-snov@100,jacc-snov@100,swjacc-snov@100,style-snov@100,tfidf-ser@100,wtfidf-ser@100,bm25-ser@100,jacc-ser@100,style-ser@100
6,dbert-ft-d1b5f,cosine,0.35,0.49254,0.8056,0.607875,0.87695,0.96101,0.3335,0.80385,0.574775,nan,nan,0.39616,0.703233,0.60135,nan,nan,0.36936,0.42961,0.37849,0.36152,0.58084,0
9,dbert-ft-6c7f3,cosine,0.4,0.49243,0.8052,0.606825,0.87682,0.96091,0.32454,0.8036,0.57405,nan,nan,0.39228,0.703433,0.60165,nan,nan,0.37636,0.42431,0.37395,0.35621,0.57584,0.01697
5,dbert-ft-71ce1,cosine,0.3,0.49234,0.8059,0.608925,0.87706,0.96109,0.3421,0.80405,0.575575,nan,0.96285,0.39998,0.703133,0.601283,nan,0.93299,0.36318,0.43259,0.38277,0.36416,0.58372,0.02627
11,dbert-ft-79517,cosine,0.45,0.49147,0.80505,0.606525,0.87676,0.96086,0.31964,0.80355,0.57395,nan,nan,0.39026,0.703633,0.602183,nan,nan,0.38056,0.42174,0.37123,0.35429,0.57352,0.02795
12,dbert-ft-2529b,cosine,0.6,0.48681,0.80485,0.60605,0.87659,0.96073,0.30766,0.8035,0.5738,0.87963,nan,0.38544,0.704467,0.603983,0.8422,nan,0.39246,0.41159,0.35853,0.34339,0.56222,0.0532
4,dbert-ft-9cd08,cosine,10,0.48658,0.8043,0.606075,0.87653,0.96063,0.3171,0.8033,0.574,nan,0.96263,0.38944,0.703767,0.603917,nan,0.93282,0.38834,0.42063,0.36649,0.35315,0.56706,0.07133
8,dbert-ft-d66a1,cosine,0.2,0.48567,0.8078,0.614,0.87757,0.96147,0.3713,0.80535,0.579725,nan,nan,0.41448,0.703733,0.60355,nan,nan,0.34572,0.43662,0.38864,0.37205,0.58737,0.09091
14,dbert-ft-833d3,cosine,3,0.485,0.80735,0.6138,0.8775,0.96138,0.37856,0.805,0.57935,nan,nan,0.41852,0.703367,0.603333,nan,nan,0.34244,0.44005,0.39358,0.37729,0.59116,0.11585
7,dbert-ft-a259b,cosine,20,0.4785,0.80435,0.6067,0.87636,0.96048,0.30234,0.80345,0.574975,0.87961,0.96258,0.38326,0.7052,0.607617,0.84219,0.93285,0.40404,0.40379,0.34783,0.33874,0.54837,0.07552
0,dbert-ft-c3c19,cosine,30,0.47426,0.8045,0.60725,0.87631,0.96044,0.29876,0.80365,0.575575,nan,nan,0.3818,0.705867,0.609167,nan,nan,0.40872,0.39796,0.34077,0.33349,0.54157,0.07907


In [ ]:
basicPrint('dbert-base')

In [ ]:
basicPrint('stylo')

In [ ]:
basicPrint('infersent')

In [ ]:
basicPrint('sent2vec')

In [ ]:
basicPrint('word2vec')

In [ ]:
basicPrint('doc2vec')

In [ ]:
basicPrint('usent')

In [ ]:
basicPrint('bert')

# All models for split version 2

In [ ]:
printAll('div', splitVersion=2)

# Combinations of models

In [ ]:
# Expected 156 rows:
basicPrint('combin')

# Deep Learning models

In [ ]:
basicPrint('DSSM_yf', noSubsampling=False)

# Reports on the test set (split version 1)

In [ ]:
basicPrint('ideal', splitVersion=1)
basicPrint('random', splitVersion=1)
basicPrint('worst', splitVersion=1)

In [8]:
printAll('div', splitVersion=1, doNormalization=True)

Normalized metrics: {'style-div@100', 'div@100', 'topic-div@100'}
serializabledict twinews-dominances initialised.


,id,model,ndcg,div@100,topic-div@100,jacc-div@100,swjacc-div@100,style-div@100
8,bm25-933f7,bm25,0.60819,0.76505,0.4908,nan,0.95773,0.5211
6,tfidf-4a1dc,tfidf,0.59695,0.71945,0.50405,0.87003,0.94874,0.4924
7,doc2vec-e013a,doc2vec,0.58202,0.7707,0.5371,0.87607,0.95831,0.5082
14,tfidf-7febb,tfidf,0.57359,0.7085,0.500225,nan,0.9455,0.48388
4,tfidf-4b89a,tfidf,0.56138,0.76055,0.567525,0.87847,0.95687,0.52766
2,nmf-6078e,nmf,0.54098,0.7959,0.410375,0.88007,0.9633,0.53528
10,dbert-ft-7847a,dbert-ft,0.53664,0.8038,0.6078,0.87635,0.96102,0.32888
15,usent-64ec0,usent,0.5172,0.8071,0.537,0.88242,0.96534,0.52854
9,lda-82272,lda,0.47486,0.80915,0.461875,0.88185,0.96634,0.5505
13,dbert-base-d092a,dbert-base,0.46352,0.7851,0.563475,0.86871,0.95755,0.48498


In [ ]:
basicPrint('combin', 'nov', splitVersion=1)